<a href="https://colab.research.google.com/github/jmarrietar/ocular/blob/master/notebooks/PAWS_DR_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Libraries

In [ ]:
# argparsers y demas Parameters

In [ ]:
#Loss functions

In [ ]:
def train_resnet():
    """
    Aqui tengo que incluir los dataloaders y demas

    definir el train_loop_fn
    """

In [ ]:
# Start training processes
def _mp_fn(rank, flags):
    global FLAGS
    FLAGS = flags
    torch.set_default_tensor_type("torch.FloatTensor")
    accuracy, data, pred, target = train_resnet()


In [ ]:
# TO DO 

if __name__ == "__main__":
    parser = init_argparse()
    FLAGS = parser.parse_args()
    print(FLAGS)
    xmp.spawn(_mp_fn, args=(FLAGS,), nprocs=FLAGS.num_cores, start_method="fork")